In [5]:
import pandas as pd

Y_df = pd.read_parquet('https://datasets-nixtla.s3.amazonaws.com/m4-hourly.parquet')

Y_df.head()

unique_id  ds      y
0        H1   1  605.0
1        H1   2  586.0
2        H1   3  586.0
3        H1   4  559.0
4        H1   5  511.0

In [7]:
uids = Y_df['unique_id'].unique()[:10] # Select 10 ids to make the example faster

Y_df = Y_df.query('unique_id in @uids') 

Y_df = Y_df.groupby('unique_id').tail(7 * 24) #Select last 7 days of data to make example faster

In [9]:
from statsforecast import StatsForecast

StatsForecast.plot(Y_df)

1. AutoARIMA: Automatically selects the best ARIMA (AutoRegressive Integrated Moving Average) model using an information criterion.

2. HoltWinters: triple exponential smoothing, Holt-Winters’ method is an extension of exponential smoothing for series that contain both trend and seasonality. 

3. SeasonalNaive: Memory Efficient Seasonal Naive predictions.

4. HistoricAverage: arthimetic mean.

5. DynamicOptimizedTheta: The theta family of models has been shown to perform well in various datasets such as M3. Models the deseasonalized time series.

In [10]:
from statsforecast.models import (
    AutoARIMA,
    HoltWinters,
    CrostonClassic as Croston, 
    HistoricAverage,
    DynamicOptimizedTheta as DOT,
    SeasonalNaive
)


# Create a list of models and instantiation parameters
models = [
    AutoARIMA(season_length=24),
    HoltWinters(),
    Croston(),
    SeasonalNaive(season_length=24),
    HistoricAverage(),
    DOT(season_length=24)
]

In [11]:
# Instantiate StatsForecast class as sf
sf = StatsForecast(
    df=Y_df, 
    models=models,
    freq='H', 
    n_jobs=-1,
    fallback_model = SeasonalNaive(season_length=7)
)

In [12]:
forecasts_df = sf.forecast(h=48, level=[90])

forecasts_df.head()

ds   AutoARIMA  AutoARIMA-lo-90  AutoARIMA-hi-90  HoltWinters  \
unique_id                                                                   
H1         749  592.461792       572.325623       612.597961        829.0   
H1         750  527.174316       495.321777       559.026855        807.0   
H1         751  488.418549       445.535583       531.301514        785.0   
H1         752  452.284454       400.677155       503.891785        756.0   
H1         753  433.127563       374.070984       492.184143        719.0   

           HoltWinters-lo-90  HoltWinters-hi-90  CrostonClassic  \
unique_id                                                         
H1               -246.367554        1904.367554       708.21405   
H1               -268.367554        1882.367554       708.21405   
H1               -290.367554        1860.367554       708.21405   
H1               -319.367554        1831.367554       708.21405   
H1               -356.367554        1794.367554       708.21405   

           SeasonalNaive  SeasonalNaive-lo-90  SeasonalNaive-hi-90  \
unique_id                                                            
H1                 635.0           537.471191           732.528809   
H1                 572.0           474.471222           669.528809   
H1                 532.0           434.471222           629.528809   
H1                 493.0           395.471222           590.528809   
H1                 477.0           379.471222           574.528809   

           HistoricAverage  HistoricAverage-lo-90  HistoricAverage-hi-90  \
unique_id                                                                  
H1              660.982117              398.03772             923.926514   
H1              660.982117              398.03772             923.926514   
H1              660.982117              398.03772             923.926514   
H1              660.982117              398.03772             923.926514   
H1              660.982117              398.03772             923.926514   

           DynamicOptimizedTheta  DynamicOptimizedTheta-lo-90  \
unique_id                                                       
H1                    592.701843                   577.677307   
H1                    525.589111                   505.449738   
H1                    489.251801                   462.072876   
H1                    456.195038                   430.554291   
H1                    436.290527                   411.051239   

           DynamicOptimizedTheta-hi-90  
unique_id                               
H1                          611.652649  
H1                          546.621826  
H1                          512.424133  
H1                          478.260956  
H1                          461.815948

In [13]:
sf.plot(Y_df,forecasts_df)

In [14]:
# Plot to unique_ids and some selected models
sf.plot(Y_df, forecasts_df, models=["HoltWinters","DynamicOptimizedTheta"], unique_ids=["H10", "H105"], level=[90])

In [15]:
# Explore other models 
sf.plot(Y_df, forecasts_df, models=["AutoARIMA"], unique_ids=["H10", "H105"], level=[90])

In [16]:
crossvaldation_df = sf.cross_validation(
    df=Y_df,
    h=24,
    step_size=24,
    n_windows=2
  )

In [19]:
crossvaldation_df.head()

ds  cutoff      y   AutoARIMA  HoltWinters  CrostonClassic  \
unique_id                                                                
H1         701     700  619.0  603.925415        847.0      742.668762   
H1         702     700  565.0  507.591736        820.0      742.668762   
H1         703     700  532.0  481.281677        790.0      742.668762   
H1         704     700  495.0  444.410248        784.0      742.668762   
H1         705     700  481.0  421.168762        752.0      742.668762   

           SeasonalNaive  HistoricAverage  DynamicOptimizedTheta  
unique_id                                                         
H1                 691.0       661.674988             612.767517  
H1                 618.0       661.674988             536.846252  
H1                 563.0       661.674988             497.824280  
H1                 529.0       661.674988             464.723236  
H1                 504.0       661.674988             440.972351

In [20]:
from datasetsforecast.losses import mse, mae, rmse


def evaluate_cross_validation(df, metric):
    models = df.drop(columns=['ds', 'cutoff', 'y']).columns.tolist()
    evals = []
    for model in models:
        eval_ = df.groupby(['unique_id', 'cutoff']).apply(lambda x: metric(x['y'].values, x[model].values)).to_frame() # Calculate loss for every unique_id, model and cutoff.
        eval_.columns = [model]
        evals.append(eval_)
    evals = pd.concat(evals, axis=1)
    evals = evals.groupby(['unique_id']).mean(numeric_only=True) # Averages the error metrics for all cutoffs for every combination of model and unique_id
    evals['best_model'] = evals.idxmin(axis=1)
    return evals

In [28]:
evaluation_df = evaluate_cross_validation(crossvaldation_df, rmse)

evaluation_df.head()

AutoARIMA  HoltWinters  CrostonClassic  SeasonalNaive  \
unique_id                                                           
H1          43.415565   211.241516      167.434204      37.697273   
H10         21.390182    53.004879       38.509552       9.836142   
H100        92.894600   347.847504      302.640259     108.287697   
H101        82.520645   166.730255      121.336662     103.826614   
H102       247.033264   481.515320      363.847961     111.601471   

           HistoricAverage  DynamicOptimizedTheta             best_model  
unique_id                                                                 
H1              144.315704              35.760849  DynamicOptimizedTheta  
H10              44.470627              19.478392          SeasonalNaive  
H100            279.912964             145.184052              AutoARIMA  
H101            131.140030             234.133728              AutoARIMA  
H102            554.725464             177.166336          SeasonalNaive

In [29]:
def get_best_model_forecast(forecasts_df, evaluation_df):
    df = forecasts_df.set_index('ds', append=True).stack().to_frame().reset_index(level=2) # Wide to long 
    df.columns = ['model', 'best_model_forecast'] 
    df = df.join(evaluation_df[['best_model']])
    df = df.query('model.str.replace("-lo-90|-hi-90", "", regex=True) == best_model').copy()
    df.loc[:, 'model'] = [model.replace(bm, 'best_model') for model, bm in zip(df['model'], df['best_model'])]
    df = df.drop(columns='best_model').set_index('model', append=True).unstack()
    df.columns = df.columns.droplevel()
    df = df.reset_index(level=1)
    return df

In [30]:
prod_forecasts_df = get_best_model_forecast(forecasts_df, evaluation_df)

prod_forecasts_df.head()

model       ds  best_model  best_model-hi-90  best_model-lo-90
unique_id                                                     
H1         749  592.701843        611.652649        577.677307
H1         750  525.589111        546.621826        505.449738
H1         751  489.251801        512.424133        462.072876
H1         752  456.195038        478.260956        430.554291
H1         753  436.290527        461.815948        411.051239

In [31]:
sf.plot(Y_df, prod_forecasts_df, level=[90])